In [1]:
import tensorflow as tf
from tensorflow import keras
# from keras.applications.resnet import ResNet50, preprocess_input
from tensorflow.keras.applications.resnet_v2 import ResNet101V2, preprocess_input
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model, load_model

from sklearn.metrics import classification_report

from os import path
import pickle
import numpy as np

import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

In [2]:
DATA_PATH = 'datasets/200x200/'

with open(path.join(DATA_PATH, 'images'), 'rb') as f:
    images = pickle.load(f)

with open(path.join(DATA_PATH, 'labels'), 'rb') as f:
    labels = pickle.load(f)

images = np.array(images)
images = preprocess_input(images)

labels = labels[:, 4:]
labels = labels.astype(int)

In [3]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [24]:
model = load_model('model_task_1.h5')
predicted = model.predict(images, verbose=0)

labels_task_1 = ['Bathroom', 'Bathroom cabinet', 'Bathroom sink', 'Bathtub', 'Bed', 'Bed frame',
                 'Bed sheet', 'Bedroom', 'Cabinetry', 'Ceiling', 'Chair', 'Chandelier', 'Chest of drawers',
                 'Coffee table', 'Couch', 'Countertop', 'Cupboard', 'Curtain', 'Dining room', 'Door', 'Drawer',
                 'Facade', 'Fireplace', 'Floor', 'Furniture', 'Grass', 'Hardwood', 'House', 'Kitchen',
                 'Kitchen & dining room table', 'Kitchen stove', 'Living room', 'Mattress', 'Nightstand',
                 'Plumbing fixture', 'Property', 'Real estate', 'Refrigerator', 'Roof', 'Room', 'Rural area',
                 'Shower', 'Sink', 'Sky', 'Table', 'Tablecloth', 'Tap', 'Tile', 'Toilet', 'Tree', 'Urban area',
                 'Wall', 'Window']

labels = np.rint(labels).astype(int)
predicted = np.rint(predicted).astype(int)

In [32]:
report = classification_report(labels, predicted, target_names=labels_task_1, output_dict=True)
report = [{**v, **{'feature': k}} for k, v in report.items()]
sorted_report = list(sorted(report, key=lambda kv: kv['f1-score']))
x =[print("'" + line['feature'] + "',", end = '') for line in sorted_report if line['f1-score'] < 0.15]

'Bathroom cabinet','Bathroom sink','Bathtub','Chair','Chandelier','Chest of drawers','Coffee table','Couch','Cupboard','Curtain','Door','Drawer','Facade','Fireplace','Grass','Hardwood','Kitchen & dining room table','Kitchen stove','Mattress','Nightstand','Refrigerator','Rural area','Shower','Sky','Tablecloth','Tree','Urban area','Window','Tap','Wall','Toilet','Dining room',

In [33]:
labels_task_1_1 = ['Bathroom cabinet','Bathroom sink','Bathtub','Chair','Chandelier','Chest of drawers','Coffee table','Couch','Cupboard','Curtain','Door','Drawer','Facade','Fireplace','Grass','Hardwood','Kitchen & dining room table','Kitchen stove','Mattress','Nightstand','Refrigerator','Rural area','Shower','Sky','Tablecloth','Tree','Urban area','Window','Tap','Wall','Toilet','Dining room']